<a href="https://colab.research.google.com/github/jinseriouspark/commerce-llm/blob/main/extract_keywords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# extract- keywords
- 진행일자 : 24.03.26
- 목적 : 해당 리뷰를 분리할 수 있는 핵심 키워드를 추출
- 비교항목 :
- 판단 기준 : 1~5를 가장 잘 분리할 수 있는 모델


In [1]:
!pip install datasets
!pip install transformers[torch]
!pip install accelrate -U
!pip install seqeval


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 902.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [2]:
from sklearn.model_selection import train_test_split
import json
#!pip install datasets
from datasets import load_dataset, load_metric
import pandas as pd

# 1. data pre-processing

In [13]:
with open('/content/drive/MyDrive/commerce-llm/rawdata/naver_shopping.txt', 'r', encoding = 'utf-8') as f:
  data =f.readlines()

In [14]:
rawdata =[i.rstrip().split('\t') for i in data]

- 20만개를 가지고 하기엔 1개 모델당 4시간 정도가 걸림
- 판단기준을 주어진 시간 내 잘 검증할 수 있도록 샘플 데이터를 아래와 같이 추출

In [15]:
import pandas as pd
raw_df = pd.DataFrame(rawdata)
raw_df.head()

features = raw_df[1]
target = raw_df[0]

raw_df[0].value_counts() / raw_df.shape[0]
# 3번은 존재하지 않음
# 1 18%, 2 32%, 4 9.3%, 5 40%

5    0.405885
2    0.319945
1    0.180240
4    0.093930
Name: 0, dtype: float64

In [17]:
# 1, 2, 4, 5 라는 라벨은 학습 불가능한 타겟이기 때문에 수정
target_dict ={v : idx for idx, v in  enumerate(raw_df[0].unique())}
raw_df[0] = raw_df[0].map(target_dict)
raw_df[0].unique()

array([0, 1, 2, 3])

In [18]:
min_cnt = int(raw_df[0].value_counts().min() * 0.1)
raw_df_for_test = raw_df.groupby(0).apply(lambda x : x.sample(min_cnt)).reset_index(drop = True)

raw_df_for_test.shape[0] / raw_df.shape[0] # 37% 만 가지고 테스트셋으로 만들어 진행?
raw_df_for_test.columns = ['target','review']
raw_df_for_test['target'] = raw_df_for_test['target'].astype(int)


X_train, X_test, _, _ = train_test_split(raw_df_for_test.values, raw_df_for_test['target'], stratify = raw_df_for_test['target'], random_state= 331)

with open('/content/drive/MyDrive/commerce-llm/rawdata/rawdata_train.jsonl','w') as f:
  for row in X_train:
    json.dump({'rating': row[0], 'review': row[1]}, f, ensure_ascii = False)
    f.write('\n')

with open('/content/drive/MyDrive/commerce-llm/rawdata/rawdata_test.jsonl','w') as f:
  for row in X_test:
    json.dump({'rating': row[0], 'review': row[1]}, f, ensure_ascii = False)
    f.write('\n')

# 2. data load

In [19]:
data_files = {'train': '/content/drive/MyDrive/commerce-llm/rawdata/rawdata_train.jsonl', 'test' : '/content/drive/MyDrive/commerce-llm/rawdata/rawdata_test.jsonl'}
review_dataset = load_dataset('json', data_files = data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [20]:
label_len = len(set(review_dataset['train']['rating']))

In [21]:
# baseline : 만약 이 모든 값들을 그냥 분류한다면 어떻게 될까?
model_name = "monologg/kobert"
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments
import torch
import torch.nn as nn
import torch.optim as optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = label_len).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
def tokenize_function(row):
  return tokenizer(row['review'], padding = 'max_length', max_length = 48, truncation = True)

# tokenize dataset
tokenized_datasets = review_dataset.map(tokenize_function, batched = True)

Map:   0%|          | 0/5634 [00:00<?, ? examples/s]

Map:   0%|          | 0/1878 [00:00<?, ? examples/s]

In [23]:
from torch.utils.data import DataLoader
tokenized_datasets = tokenized_datasets.rename_column('rating','labels')
tokenized_datasets.set_format('torch', columns = ['input_ids','attention_mask','token_type_ids','labels'])
#train_dataloader = DataLoader(tokenized_datasets['train'], shuffle = True, batch_size = 4)
#test_dataloader = DataLoader(tokenized_datasets['test'], batch_size = 4)


In [24]:
optimizer = optim.AdamW(model.parameters(), lr = 5e-5)
criterion = nn.CrossEntropyLoss()

In [ ]:
#!pip install seqeval
#!pip install datasets
from datasets import load_dataset, load_metric
import numpy as np

metric = load_metric('seqeval')
def compute_metrics(pred):
  predictions, labels = pred
  print(predictions)
  predictions = np.argmax(predictions, axis=1)

  # speical token은 삭제
  true_predictions = [
      [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(predictions, labels)
  ]

  true_labels = [
      [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(predictions, labels)
  ]

  # metrics.compute
  results = metric.compute(predictions = true_predictions, references = true_labels)
  flattened_results = {
        "overall_precision": results["overall_precision"],
        "overall_recall": results["overall_recall"],
        "overall_f1": results["overall_f1"],
        "overall_accuracy": results["overall_accuracy"],
  }

  for k in results.keys():
    if k not in flattened_results.keys():
      flattened_results[f'{k}_f1'] = results[k]['f1']

  return flattened_results

In [26]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer)

In [121]:
training_args = TrainingArguments(
    output_dir = '/content/drive/MyDrive/commerce-llm/model/kobert_finetuning_reviewscore_cls',
    evaluation_strategy = 'steps',
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 7,
    weight_decay = 0.01,
    logging_steps = 500,
    save_strategy = 'no',
    use_cpu = False
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['test'],
    data_collator = data_collator,
    tokenizer = tokenizer,

)

AttributeError: 'KeyBERT' object has no attribute 'to'

In [43]:
trainer.train()

Step,Training Loss,Validation Loss
500,1.222400,1.317287
1000,1.167200,1.307832
1500,1.143700,1.364739
2000,1.055100,1.401480


TrainOutput(global_step=2471, training_loss=1.1210835980577827, metrics={'train_runtime': 396.4973, 'train_samples_per_second': 99.466, 'train_steps_per_second': 6.232, 'total_flos': 972821262696192.0, 'train_loss': 1.1210835980577827, 'epoch': 7.0})

In [57]:
predict_result = trainer.predict(tokenized_datasets['test'])
predict_result.predictions, predict_result.label_ids

(array([[-0.8992183 ,  1.6183875 ,  0.5778197 , -0.73134357],
        [-0.17310043, -0.2012808 , -0.24610603, -0.06574325],
        [ 0.1996027 ,  0.07650811,  0.54582673, -1.0155596 ],
        ...,
        [ 0.9108193 , -0.4345903 , -2.0305946 ,  1.433141  ],
        [-0.21814449, -0.1103323 , -0.18175657, -0.10017163],
        [ 0.06642102,  1.1977665 , -1.696976  ,  0.75413847]],
       dtype=float32),
 array([2, 3, 3, ..., 2, 1, 1]))

In [61]:
from sklearn.metrics import classification_report
output_dict = classification_report(np.argmax(predict_result.predictions, axis=1), predict_result.label_ids, output_dict = True)
pd.DataFrame(output_dict).T

,precision,recall,f1-score,support
0,0.334043,0.422043,0.372922,372.000000
1,0.336170,0.325103,0.330544,486.000000
2,0.332623,0.376812,0.353341,414.000000
3,0.420043,0.325083,0.366512,606.000000
accuracy,0.355698,0.355698,0.355698,0.355698
macro avg,0.355720,0.362260,0.355830,1878.000000
weighted avg,0.362031,0.355698,0.355570,1878.000000


# 비교 모델
- [참고자료](https://https://www.analyticsvidhya.com/blog/2022/03/keyword-extraction-methods-from-documents-in-nlp/)

1. Roberta : https://www.mdpi.com/2079-9292/12/22/4560
2. Spacy
3. Textrank
4. KeyBert
5. applied NLI

In [ ]:
# KEYWORD EXTRACTION BY keybert

# KEYWORD EXTRACTION BY keybert
- keybert 란?
  - CountVectorizer 를 사용하여 문장 속 keyword, keyphrase  후보를 만든다
  - Language model을 사용해서 문장과 키워드를 각각 임베딩 한다 (sentence-transformers 등을 사용)
  - 인풋 문장과 후보 키워들 간의 코사인ㅇ 유사도를 계산하고, 가장 큰 유사도부터 '키워드' 라고 지정한다

In [ ]:
!pip install keybert
# !pip install keybert[flair]
# !pip install keybert[gensim]
# !pip install keybert[spacy]
# !pip install keybert[use]

In [69]:
from keybert import KeyBERT
from tqdm import tqdm

model = KeyBERT(model="all-MiniLM-L6-v2")
keywords_list = []
for raw in tqdm(review_dataset['train'][:]['review']):
  keywords = model.extract_keywords(raw)
  keywords_list.append(keywords)

100%|██████████| 5634/5634 [02:22<00:00, 39.49it/s]


In [70]:
len(keywords_list)

5634

In [74]:
import pandas as pd
result_df = pd.DataFrame(data = {'origin' : review_dataset['train'][:]['review'], 'keywords': keywords_list})

In [76]:
result_df.head()
# 문제점 : 띄어쓰기가 반영되지 않아 '색상도화사하고' 라는 것이 하나의 keyphrase 처럼 등장
# 해결방안 : 비지도 학습 띄어쓰기 모델 생성하여 전처리 시 적용

,origin,keywords
0,별루에여 지직 거려여ㅠ,"[(지직, 0.7013), (별루에여, 0.6623), (거려여ㅠ, 0.402)]"
1,진짜 별로에요 머리털다 뽑힐거같아요,"[(머리털다, 0.775), (별로에요, 0.7453), (뽑힐거같아요, 0.395..."
2,사이즈가 너무 작네요. 알감자 수준 그래도 싱싱하고 맛은 있어요. 첫날 받아서 감자...,"[(손질하기, 0.7494), (사이즈가, 0.7031), (싱싱하고, 0.6975..."
3,색상도화사하고 좋아요ㆍ,"[(색상도화사하고, 0.9869), (좋아요ㆍ, 0.2959)]"
4,얌얌 손님 온다해서 주문한 양고기! 손님들은 처음 맛 본 양고기! 매우 만족하시면서...,"[(손님들은, 0.7852), (만족하시면서, 0.7063), (온다해서, 0.69..."


## guided KeyBERT
- 기존 KeyBERT는 문장 내 중요도를 기반으로 키워드를 추출
- guided KeyBERT가 있으면, 해당 키워드와의 유사도를 구해 키워드를 추출할 수 있음

In [88]:
from keybert import KeyBERT
kw_model = KeyBERT()

# 카테고리 별로 추출하고 있는 키워드가 상이함
# 네이버의 경우 키워드를 눌렀을 때, 관련 키워드가 적힌 문장에 줄을 쳐서 보여줌

seed_keywords = ['배송']
keywords = kw_model.extract_keywords(review_dataset['train']['review'], seed_keywords = seed_keywords)

In [89]:
result_df['keywords_배송']= keywords

In [96]:
# 배송과 관련된 키워드의 score 가 높아진 것을 확인할 수 있음
# 그러나 배송 관련된 이슈 발생 여부를 해당 방식으로만 찾기에는 한계가 존재
result_df[result_df['origin'].str.contains('배송')].head(50)

,origin,keywords,keywords_배송
10,추석 연휴 제외하고 배송도 빨랐고 유통기한도 넉넉한 트위즐러 잘 받았습니다!,"[(유통기한도, 0.6946), (연휴, 0.485), (배송도, 0.4608), ...","[(유통기한도, 0.7776), (배송도, 0.6503), (연휴, 0.5532),..."
18,배송이 너무 너무 느려요.ㅠ 바지가 미끄러져서 자꾸 떨어지네요.,"[(배송이, 0.7324), (바지가, 0.6364), (너무, 0.6334), (...","[(배송이, 0.8389), (바지가, 0.7016), (너무, 0.6413), (..."
23,깔끔포장으로 배송 잘 받았습니다.,"[(배송, 0.8165), (받았습니다, 0.4963), (깔끔포장으로, 0.4963)]","[(배송, 0.8937), (받았습니다, 0.4894), (깔끔포장으로, 0.4894)]"
38,배송이 일주일이나 걸림,"[(걸림, 0.7244), (배송이, 0.6805), (일주일이나, 0.6251)]","[(배송이, 0.8153), (걸림, 0.7432), (일주일이나, 0.6538)]"
45,배송도 빠르고 애기도 잘 무네요~,"[(배송도, 0.6752), (무네요, 0.6652), (애기도, 0.655), (...","[(배송도, 0.7927), (애기도, 0.7002), (무네요, 0.6309), ..."
48,져지는 정말좋은데 배송이 정말 완전 꽝이네요... 주문한지 일주일지나서야 도착했습니...,"[(의류출시하시면서, 0.7572), (배송건이, 0.6877), (당연히, 0.6...","[(배송건이, 0.8079), (의류출시하시면서, 0.7887), (배송이, 0.7..."
61,배송은 빨랐으나.. 털이 정말 많이빠지네요 입에도 얼굴에도 온사방에 붙고 들어가고 ㅠㅠ,"[(배송은, 0.5945), (온사방에, 0.5887), (입에도, 0.505), ...","[(배송은, 0.7646), (온사방에, 0.7027), (얼굴에도, 0.5971)..."
70,재구매 여러번 했어요 배송 빠르고 좋아요,"[(여러번, 0.6852), (배송, 0.6669), (재구매, 0.6463), (...","[(배송, 0.8019), (여러번, 0.7235), (재구매, 0.6529), (..."
92,배송도 빠르고 처음 써보는 클렌져 제형이라 버블이 너무 신기하네요 당김은 없지만 촉...,"[(제형이라, 0.5869), (클렌져, 0.499), (버블이, 0.4805), ...","[(제형이라, 0.6814), (버블이, 0.6216), (배송도, 0.6215),..."
114,화력도 좋고 디자인도 깔끔하고 이뻐요~ 배송도 빠름빠름빠흠,"[(화력도, 0.8317), (디자인도, 0.7647), (배송도, 0.6636),...","[(화력도, 0.8239), (배송도, 0.7877), (디자인도, 0.7684),..."


In [120]:
result_df.to_csv(f'/content/drive/MyDrive/commerce-llm/results/result_{seed_keywords[0]}.csv')

# KeyLLM with OpenAI

- 모델 준비하기
- prompt 결과물 정리하기

In [118]:
!pip install openai
!pip install tiktoken


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.1 MB/s eta 0:00:00


In [ ]:
# 참고자료 : https://platform.openai.com/docs/tutorials/web-qa-embeddings
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')


In [117]:
import openai
from keybert.llm import OpenAI
from keybert import KeyLLM

# Create your LLM
client = openai.OpenAI(api_key=MY_API_KEY)
llm = OpenAI(client)

# Load it in KeyLLM
kw_model = KeyLLM(llm)

# Extract keywords
document = "The website mentions that it only takes a couple of days to deliver but I still have not received mine."
keywords = kw_model.extract_keywords(document)

ModuleNotFoundError: In order to use OpenAI you will need to install via;

`pip install openai` 



In [116]:
import openai
from keybert.llm import OpenAI
from keybert import KeyLLM

# create llm
with open('/content/drive/MyDrive/commerce-llm/env/openai_apikey_kimseopp.txt', 'r') as f:
  openai.api_key = f.readline().strip()

In [111]:
criteria_1_representativeness = """
Human: You will be given all reviews and the keywords of reviews.
Your task is to rate the keywords on criteria.
Please make sure you read and understand criteria and steps carefully. Please keep this document open while reviewing, and refer to it as needed.

<criteria>
Representativeness (1-5) - the extent to which the keywords captures key words across the entirety of the reviews, demonstrating a comprehensive understanding.
</criteria>

<steps>
Step 1. Carefully read the reviews and keywords provided.
Step 2. Evaluate how representative the keywords is across the entirety of the reviews, considering coherence.
Step 3. Rate the representativeness on a scale of 1 to 5:
    1 - Insignificantly Representative: The keywords fails to capture key words from the reviews, lacking a comprehensive understanding of the content.
    2 - Barely Representative: The keywords includes only a few relevant keywords, with significant gaps in understanding the reviews.
    3 - Moderately Representative: The keywords captures some key words, but misses others, resulting in a partial understanding of the reviews.
    4 - Largely Representative: The keywords effectively includes relevant keywords, demonstrating a good understanding of the reviews, although some areas may be lacking.
    5 - Fully Representative: The keywords comprehensively captures key words from the reviews, exhibiting a thorough understanding of the content.
Step 4. Provide the criteria rating from 1 to 5 based solely on the provided criteria.
Step 5. 4. Provide specific feedback on the issues you identified and suggestions for improvement. Be constructive and professional.
Step 6. Submit your criteria score and feedback comments to appropriate team member for consideration.
</steps>

<output_format>
The output is provided in json format and has `thoughts`, `criteria_rating`, and `feedback` as keys.
For detailed output format description, please check the provided json format.

{json_schema}
{'thoughs': 'your_thought_',
'criteria_rating' : 1,
'feedback' : 'your_feedback'
}
"""


In [ ]:
example_prompt = """ """

prompt = example_prompt + keyword_prompt

In [112]:
llm = TextGeneration( prompt = criteria_1_representativeness)
kw_model = KeyLLM(llm)

TypeError: TextGeneration.__init__() missing 1 required positional argument: 'model'

In [ ]:
aa

# KeyLLM with KeyBERT + Mistral 7b

In [97]:
%%capture
!pip install --upgrade git+https://github.com/UKPLab/sentence-transformers
!pip install keybert ctransformers[cuda]
!pip install --upgrade git+https://github.com/huggingface/transformers

In [ ]:
from ctransforemrs import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    ''
)

In [ ]:
!git lfs install
!git clone https://huggingface.co/mistralai/Mistral-7B-v0.1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q torch datasets
!pip install -q accelerate==0.21.0
!pip install -q peft==0.4.0
!pip install -qbitsandbytes==0.40.2
!pip install -q transformers==4.31.0
!pip install -q trl==0.4.7

In [ ]:
!nvidia-smi

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

In [ ]:
model_path= '/content/Mistral-7B-v0.1'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path,
                                             trust_remote_code = True,
                                             device_map = 'auto',
                                             load_in_8bit = True,
                                             max_memory = {0:"15GB"})


In [ ]:
pip install accelerate

In [ ]:
model.eval()